In [1]:
!pip install -q py-feat # Instalamos la librería py-feat

In [2]:
import os
import errno
import cv2
import glob

from feat.detector import Detector

from matplotlib import pyplot as plt

from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [12]:
# Funciones.

def process_data(path, size): # Default
    print('Procesando datos...')
    
    #-Airán: Contador de elementos procesados
    elementos_procesados = 0
    
    #-Airan: Información y muestra 1º elemento
    first = True
    result_1 = 'NaN'
    fex_1 = 'NaN'
    
    # Instancia del detector con modelos por defecto
    #detector = Detector()
    detector = Detector(emotion_model = "fer") # Modelo fer
    
    sub_paths = ['Train', 'Validation'] #-Airan: Sub_Path
    
    # Carpetas con muestras
    folders = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    
    # Etiquetas del clasificador
    switch_labels = [0, 1, 2, 3, 6, 4, 5] # indice = classlabels ; value = py_feat_labels
    classlabels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'] # Folders
    py_feat_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise' , 'neutral'] # Py-feat
    
    y = []
    y_pred = []
    
    y_train = []
    y_pred_train = []
    
    y_validation = []
    y_pred_validation = []
    
    n_clases = 0
    for fex in folders:
        
        for sub_path in sub_paths: #-Airan: Obtener todo el dataset: Train, Validation
            
            folder = os.path.join((path + sub_path), fex)
            
            for file_name in os.listdir(folder):

                # Asume imágenes en formato png
                if file_name.endswith('.png'):
                    
                    elementos_procesados += 1
                    
                    #-Airan: Ruta completa imagen, "detect_image" sólo admite path.
                    abs_path = os.path.join(folder, file_name)
                    
                    #-- Código
                    # FEX de imagen
                    result = detector.detect_image(abs_path)
                    # Obtiene el dataframe con únicamente informacion de emociones
                    fex_ = result.emotions()
                    #print(fex_)

                    if fex_.iloc[0].isnull().all() == False: # Garantizamos detección de caras.
                        #-Airán: Información y muestra 1º elemento
                        if first:
                            print('Categoría Predicha:',fex_.idxmax(axis=1).tolist()[0])
                            result.plot_detections()
                            first = False
                        
                        #--- Conjunto completo.
                        y.append(switch_labels[n_clases])
                        y_pred.append(py_feat_labels.index(fex_.idxmax(axis=1).tolist()[0]))
                        #--- Conjunto Train.
                        if sub_path == sub_paths[0]:
                            y_train.append(switch_labels[n_clases])
                            y_pred_train.append(py_feat_labels.index(fex_.idxmax(axis=1).tolist()[0]))
                        #--- Conjunto Validation.
                        if sub_path == sub_paths[1]:
                            y_validation.append(switch_labels[n_clases])
                            y_pred_validation.append(py_feat_labels.index(fex_.idxmax(axis=1).tolist()[0]))
                        
        n_clases += 1
    
    print('\nINFORMACIÓN DATASET')
    print('Número de carpetas:', len(folders))
    print('Número de clases:', len(py_feat_labels))
    print('Tamaño de la imagen:',size)
    print('Elementos procesados:',elementos_procesados)
    
    print('FIN Procesando datos.')
    return y, y_pred, y_train, y_pred_train, y_validation, y_pred_validation

In [13]:
#--- EXPERIMENTACIÓN ---
#--- Tamaños de imágenes: (143, 181), (71, 90), (35, 45), (17, 22)
#--- DataSet: (143, 181)
switch_labels = [0, 1, 2, 3, 6, 4, 5] # indice = classlabels ; value = py_feat_labels
py_feat_labels = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise' , 'neutral'] # Py-feat

print('--- INICIO SIMULACIÓN ---')

#------------------------ SIMULACION 1 -------------
print('\nSimulación: 1')
y, y_pred, y_train, y_pred_train, y_validation, y_pred_validation = process_data('../input/sfewnew/SFEW/', (143, 181))
#--- Información de los resultados obtenidos:
print("\nPy-Feat FER Metrics") # Reconocimiento Facial de Emociones.
if y == []:
    print('No se han detectado caras en el data-set dado.')
else:
    #--- Información Conjunto Completo:
    print("\nInformación Conjunto Completo")
    precision_score(y, y_pred, average='weighted')
    recall_score(y, y_pred, average='weighted')
    print(classification_report(y, y_pred, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y, y_pred, labels=range(len(switch_labels))))
    #--- Información Conjunto Train:
    print("\nInformación Conjunto Train")
    precision_score(y_train, y_pred_train, average='weighted')
    recall_score(y_train, y_pred_train, average='weighted')
    print(classification_report(y_train, y_pred_train, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_train, y_pred_train, labels=range(len(switch_labels))))
    #--- Información Conjunto Validation:
    print("\nInformación Conjunto Validation")
    precision_score(y_validation, y_pred_validation, average='weighted')
    recall_score(y_validation, y_pred_validation, average='weighted')
    print(classification_report(y_validation, y_pred_validation, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_validation, y_pred_validation, labels=range(len(switch_labels))))

#------------------------ SIMULACION 2 -------------
print('\nSimulación: 2')
y, y_pred, y_train, y_pred_train, y_validation, y_pred_validation = process_data('../input/sfewnew/SFEW_17x22/', (17, 22))
#--- Información de los resultados obtenidos:
print("\nPy-Feat FER Metrics") # Reconocimiento Facial de Emociones.
if y == []:
    print('No se han detectado caras en el data-set dado.')
else:
    #--- Información Conjunto Completo:
    print("\nInformación Conjunto Completo")
    precision_score(y, y_pred, average='weighted')
    recall_score(y, y_pred, average='weighted')
    print(classification_report(y, y_pred, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y, y_pred, labels=range(len(switch_labels))))
    #--- Información Conjunto Train:
    print("\nInformación Conjunto Train")
    precision_score(y_train, y_pred_train, average='weighted')
    recall_score(y_train, y_pred_train, average='weighted')
    print(classification_report(y_train, y_pred_train, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_train, y_pred_train, labels=range(len(switch_labels))))
    #--- Información Conjunto Validation:
    print("\nInformación Conjunto Validation")
    precision_score(y_validation, y_pred_validation, average='weighted')
    recall_score(y_validation, y_pred_validation, average='weighted')
    print(classification_report(y_validation, y_pred_validation, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_validation, y_pred_validation, labels=range(len(switch_labels))))

#------------------------ SIMULACION 3 -------------
print('\nSimulación: 3')
y, y_pred, y_train, y_pred_train, y_validation, y_pred_validation = process_data('../input/sfewnew/SFEW_35x45/', (35, 45))
#--- Información de los resultados obtenidos:
print("\nPy-Feat FER Metrics") # Reconocimiento Facial de Emociones.
if y == []:
    print('No se han detectado caras en el data-set dado.')
else:
    #--- Información Conjunto Completo:
    print("\nInformación Conjunto Completo")
    precision_score(y, y_pred, average='weighted')
    recall_score(y, y_pred, average='weighted')
    print(classification_report(y, y_pred, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y, y_pred, labels=range(len(switch_labels))))
    #--- Información Conjunto Train:
    print("\nInformación Conjunto Train")
    precision_score(y_train, y_pred_train, average='weighted')
    recall_score(y_train, y_pred_train, average='weighted')
    print(classification_report(y_train, y_pred_train, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_train, y_pred_train, labels=range(len(switch_labels))))
    #--- Información Conjunto Validation:
    print("\nInformación Conjunto Validation")
    precision_score(y_validation, y_pred_validation, average='weighted')
    recall_score(y_validation, y_pred_validation, average='weighted')
    print(classification_report(y_validation, y_pred_validation, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_validation, y_pred_validation, labels=range(len(switch_labels))))

#------------------------ SIMULACION 4 -------------
print('\nSimulación: 4')
y, y_pred, y_train, y_pred_train, y_validation, y_pred_validation = process_data('../input/sfewnew/SFEW_71x90/', (71, 90))
#--- Información de los resultados obtenidos:
print("\nPy-Feat FER Metrics") # Reconocimiento Facial de Emociones.
if y == []:
    print('No se han detectado caras en el data-set dado.')
else:
    #--- Información Conjunto Completo:
    print("\nInformación Conjunto Completo")
    precision_score(y, y_pred, average='weighted')
    recall_score(y, y_pred, average='weighted')
    print(classification_report(y, y_pred, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y, y_pred, labels=range(len(switch_labels))))
    #--- Información Conjunto Train:
    print("\nInformación Conjunto Train")
    precision_score(y_train, y_pred_train, average='weighted')
    recall_score(y_train, y_pred_train, average='weighted')
    print(classification_report(y_train, y_pred_train, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_train, y_pred_train, labels=range(len(switch_labels))))
    #--- Información Conjunto Validation:
    print("\nInformación Conjunto Validation")
    precision_score(y_validation, y_pred_validation, average='weighted')
    recall_score(y_validation, y_pred_validation, average='weighted')
    print(classification_report(y_validation, y_pred_validation, target_names=py_feat_labels))
    print("Confussion matrix:")
    print(confusion_matrix(y_validation, y_pred_validation, labels=range(len(switch_labels))))

print('--- FIN SIMULACIÓN ---')